In [54]:
#imports
import time
import qcportal as ptl
import numpy as np
import matplotlib.pyplot as plt
from fragmenter.utils import HARTREE_2_KJMOL
from simtk import unit
import os, sys
from fragmenter import chemi
import cmiles
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


In [92]:
#functions 

def torsion_barrier_for_molecule(tdr_object, smiles, show_plots=False):
    """
    Takes in a single torsion drive record that has energies from multiple conformers (at different torsion angles), evaluates the torsion barrier 
    
    Parameters
    ----------
    tdr_object : object
        torsion drive record from QC archive for a molecule
    
    Returns
    -------
    tdr_object.id : int
        id of the TD record
    dihedral_indices: list
        list of atom indices for which torsion is driven in this record
    torsion_barrier: float
        torsion barrier energy in KJ/mol, maximum of all the barriers
    mol: oemol object
        oemol from the smiles in dataframe index
    """
    energies = list(tdr_object.get_final_energies().values())
    tmp = list(tdr_object.get_final_energies().keys())
    angles = [i[0]*np.pi/180 for i in tmp]
    angles, energies = zip(*sorted(zip(angles, energies)))
    angles = np.array(angles)
    energies = np.array(energies)
    angles = np.append(angles[-3:] - 2*np.pi, np.append(angles, angles[:3] + 2*np.pi))
    energies = np.append(energies[-3:], np.append(energies, energies[:3]))
    
    idx = []
    for i in range(len(angles)-2):
        m1 = (energies[i+1]-energies[i])/(angles[i+1]-angles[i])
        m2 = (energies[i+2]-energies[i+1])/(angles[i+2]-angles[i+1])
        if np.sign(m1) == np.sign(m2):
            continue
        else:
            idx.append(i+1)
        
    if (show_plots):
        min_ener = min(energies)
        energies_y = (energies-min_ener)*HARTREE_2_KJMOL
        fontsize = 14
        plt.figure()
        plt.plot(angles*180/np.pi, energies_y, 'b-X', angles[idx]*180/np.pi, energies_y[idx], 'ro')
        plt.legend(['QM data', 'Max, min'], bbox_to_anchor=(1, 1), fontsize=fontsize)
        plt.title('Torsion drive interpolation', fontsize=fontsize)
        plt.xlabel('Dihedral Angle [Degrees]', fontsize=fontsize)
        plt.ylabel('Relative energy [KJ / mol]', fontsize=fontsize)
        plt.xticks(fontsize=fontsize)
        plt.yticks(fontsize=fontsize)
        fig_name = 'plot_' + tdr_object.id + '.png'
        plt.savefig(fig_name)
        plt.show()
        
    torsion_barriers = []
    for i in range(int(len(idx)-1)):
        torsion_barriers.append(abs(HARTREE_2_KJMOL*abs(energies[idx[i]] - energies[idx[i+1]])))
    torsion_barriers = np.array(torsion_barriers)
    
    # get dihedral indices and pass on to get_wbo function
    dihedral_indices = list(tdr_object.dict()['keywords']['dihedrals'][0])
    mol = chemi.smiles_to_oemol(smiles)
    mol.SetData("TB", max(torsion_barriers))
    mol.SetData("TDindices", dihedral_indices)
    mol.SetData("TDid", tdr_object.id)
    mol.SetData("cmiles", smiles)
    return mol
    #return(tdr_object.id, dihedral_indices, max(torsion_barriers), mol)


def analyzeTorsion(tID, ff, k1, k2, qcaDS, fileName):
        """
        Description:
        This function takes in a torsion id, and performs analysis on it using a QCA torsion drive dataset
        This function will generate a plot with a .pdf extension

        Input:
        tID: String of a torsion ID from an .offxml format force field e.g. "t43"
        ff: file name of a .offxml style force field
        qcaDS: String of a name of a QCA torsion drive dataset
        fileName: String for the .pdf file name of the plot of WBO versus torsion drive barrier
        k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
        k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

        Return:
        resultDict: A dictionary of the results from the residual calculation. The key is the smiles and the
        items is a list of the [torsion barrier hieght, WBO, residual].
        """

        #WIP Load in the QCA dataset, convert the dataset into OEMols with data tags containing WBO and torsion barrier height
        molList = loadDataset(qcaDS)


        molsTorsion=[]
        for mol in molList:
            status = checkTorsion(tID, ff, mol)
            if status == True:
                molsTorsion.append(mol)

        resultDict={}

        for m in molsTorsion:
            resultDict[OEMolsToSmiles(m)]= [m.GetData("TorsionBarrier"), m.GetData("WBO"), getResidual(m, k1, k2)]

        plotResults(resultDict, fileName, k1, k2)


        return resultDict

def loadDataset(datasets):
    """
        To load torsion drive datasets and return a list of molecules with torsion barriers and associated wiberg bond order
        
        Parameters
        ----------
        datasets : list of tuples containing the dataset name and specification
        
        Returns
        -------
        molList : each row contains the tdr_object.id, oemol_object, dihedral_indices, torsion_barrier, wbo
    """
    molList = []
    for dataset_tuple in datasets:
        print("Reading dataset: ", dataset_tuple[0], ", and specification: ", dataset_tuple[1])
        molList.extend(loadDataset_low(dataset_tuple[0], dataset_tuple[1]))

    return molList
    
    

    

def loadDataset_low(datasetName, specification):
    """
    Low level call to load each torsion drive dataset and return a list of molecules
    
        Parameters
        ----------
        datasetName : str
            torsion drive dataset name.  
        specification : str
            specification in the dataset. Example: "B3LYP-D3", "default", "UFF" 
        
        Returns
        -------
        molList : list of objects
            each row contains the tdr_object.id, dihedral_indices, torsion_barrier, oemol_object
    """
    while True:    
        try:
            assert(datasetName)
            break
        except AssertionError:
            print ("datasetName is empty. Check input list of dataset tuples")
            raise
    while True:    
        try:
            assert(specification)
            break
        except AssertionError:
            print ("specification is empty. Check input list of dataset tuples")
            raise

    # initiate qc portal instance
    client = ptl.FractalClient()    
    # from the TorsionDriveDataset collection picking up given datasetName
    ds = client.get_collection("TorsionDriveDataset", datasetName)
    ds.status([specification], status="COMPLETE")
 
    # Serial implementation
    tb = []
    for i in range(ds.df.size):
        if (ds.df.iloc[i,0].status == 'COMPLETE'):
            smiles = ds.df.index[i]
            cmiles_ids = cmiles.get_molecule_ids(smiles, strict=False)
            mapped_smiles = cmiles_ids['canonical_isomeric_explicit_hydrogen_mapped_smiles']
            tb.append(torsion_barrier_for_molecule(ds.df.iloc[i, 0], mapped_smiles))
    print("Total records processed for this dataset:", len(tb), "out of ", len(ds.df))
    return(tb)





def getResidual(oemol, k1, k2):
    """
    Description:
    Finds the residual of an oemol by taking the input k1 and k2 values and finding the y=mx+b form and calculates
    the difference from the wbo, torsion barrier height point

    input:
    oemol: OEMol from oechem, contains datatags for WBO and torsion barrier height
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion
    """
    #WIP

    return residual

def plotResult(resultDict, filename, k1, k2):
    """
    Description:
    Creates a scatter plot of WBO versus the torsion barrier height with the y=mx + b form of the torsion interpolation.
    Saves plot to .pdf

    Input:
    resultDict: Dictionary of the data for plotting. The keys are smiles, and the items are a list of WBO, torsion barrier height
    and residual
    fileName: Name of file to save .pdf form of the plot
    k1: Float of the force constant for wiberg bond order equal to 1 for the specified torsion
    k2: Float of the force constant for wiberg bond order equal to 2 for the specified torsion

    return:
    """
    #WIP
    return


def checkTorsion(molList, TID):
    matches=[]
    for mol in molList: 
        #molecule = Molecule(mol, allow_undefined_stereo=True)
        molecule = Molecule.from_mapped_smiles(mol.GetData("cmiles"))
        #topology=Topology._from_openeye(mol)
        topology = Topology.from_molecules([molecule])
        # Let's label using the Parsley force field
        forcefield = ForceField('openff_unconstrained-1.2.1.offxml')
        # Run the molecule labeling
        molecule_force_list = forcefield.label_molecules(topology)
        params=[]
        # Print out a formatted description of the torsion parameters applied to this molecule
        for mol_idx, mol_forces in enumerate(molecule_force_list):
            #print(f'Forces for molecule {mol_idx}')
            for force_tag, force_dict in mol_forces.items():
                if force_tag == 'ProperTorsions':
                    #print(f"\n{force_tag}:")
                    
                    for (atom_indices, parameter) in force_dict.items():
                        #atomstr=''
                        atomList=[]
                        for idx in atom_indices:
                            atomList.append(idx)
                            #atomstr += '%3s' % idx
                        print(atomList)
                        print(mol.GetData("TDindices"))
                        params.append(parameter.id)
                        if atomList == mol.GetData("TDindices"):
                            print("Match!")
                            matches.append(mol)
                        #print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )
    return matches, params


In [93]:
datasetName="OpenFF Gen 2 Torsion Set 5 Bayer"
#datasetName="OpenFF Rowley Biaryl v1.0"
molList=loadDataset_low(datasetName, "default")
#print(molList)

#mol.SetData("TB", max(torsion_barriers))
#    mol.SetData("TDindices", dihedral_indices)
#    mol.SetData("TDid", tdr_object.id)
    
#for mol in molList:
    #print(mol.GetData("TDindices"))

mols=checkTorsion(molList, 't69')
print(mols)





Total records processed for this dataset: 77 out of  100
[0, 2, 9, 3]
[10, 14, 15, 22]
[0, 2, 9, 24]
[10, 14, 15, 22]
[0, 8, 1, 3]
[10, 14, 15, 22]
[0, 8, 1, 26]
[10, 14, 15, 22]
[0, 8, 23, 14]
[10, 14, 15, 22]
[1, 3, 9, 2]
[10, 14, 15, 22]
[1, 3, 9, 24]
[10, 14, 15, 22]
[1, 8, 0, 2]
[10, 14, 15, 22]
[1, 8, 0, 25]
[10, 14, 15, 22]
[1, 8, 23, 14]
[10, 14, 15, 22]
[2, 0, 8, 23]
[10, 14, 15, 22]
[2, 9, 3, 28]
[10, 14, 15, 22]
[3, 1, 8, 23]
[10, 14, 15, 22]
[3, 9, 2, 27]
[10, 14, 15, 22]
[4, 16, 6, 20]
[10, 14, 15, 22]
[4, 16, 6, 31]
[10, 14, 15, 22]
[4, 18, 20, 6]
[10, 14, 15, 22]
[4, 18, 20, 12]
[10, 14, 15, 22]
[5, 17, 7, 21]
[10, 14, 15, 22]
[5, 17, 7, 32]
[10, 14, 15, 22]
[5, 19, 21, 7]
[10, 14, 15, 22]
[5, 19, 21, 13]
[10, 14, 15, 22]
[6, 16, 4, 18]
[10, 14, 15, 22]
[6, 16, 4, 29]
[10, 14, 15, 22]
[6, 20, 12, 15]
[10, 14, 15, 22]
[6, 20, 12, 39]
[10, 14, 15, 22]
[6, 20, 12, 40]
[10, 14, 15, 22]
[7, 17, 5, 19]
[10, 14, 15, 22]
[7, 17, 5, 30]
[10, 14, 15, 22]
[7, 21, 13, 15]
[10, 14, 1

[0, 1, 3, 5]
[24, 7, 11, 31]
[0, 1, 3, 20]
[24, 7, 11, 31]
[0, 2, 4, 5]
[24, 7, 11, 31]
[0, 2, 4, 21]
[24, 7, 11, 31]
[1, 0, 2, 4]
[24, 7, 11, 31]
[1, 0, 2, 19]
[24, 7, 11, 31]
[1, 3, 5, 4]
[24, 7, 11, 31]
[1, 3, 5, 8]
[24, 7, 11, 31]
[2, 0, 1, 3]
[24, 7, 11, 31]
[2, 0, 1, 18]
[24, 7, 11, 31]
[2, 4, 5, 3]
[24, 7, 11, 31]
[2, 4, 5, 8]
[24, 7, 11, 31]
[3, 1, 0, 17]
[24, 7, 11, 31]
[3, 5, 4, 21]
[24, 7, 11, 31]
[3, 5, 8, 13]
[24, 7, 11, 31]
[3, 5, 8, 25]
[24, 7, 11, 31]
[3, 5, 8, 26]
[24, 7, 11, 31]
[4, 2, 0, 17]
[24, 7, 11, 31]
[4, 5, 3, 20]
[24, 7, 11, 31]
[4, 5, 8, 13]
[24, 7, 11, 31]
[4, 5, 8, 25]
[24, 7, 11, 31]
[4, 5, 8, 26]
[24, 7, 11, 31]
[5, 3, 1, 18]
[24, 7, 11, 31]
[5, 4, 2, 19]
[24, 7, 11, 31]
[5, 8, 13, 9]
[24, 7, 11, 31]
[5, 8, 13, 10]
[24, 7, 11, 31]
[6, 9, 13, 8]
[24, 7, 11, 31]
[6, 9, 13, 10]
[24, 7, 11, 31]
[6, 16, 11, 7]
[24, 7, 11, 31]
[6, 16, 11, 31]
[24, 7, 11, 31]
[6, 16, 11, 32]
[24, 7, 11, 31]
[8, 5, 3, 20]
[24, 7, 11, 31]
[8, 5, 4, 21]
[24, 7, 11, 31]
[8, 13, 9, 

[0, 2, 4, 1]
[32, 24, 19, 55]
[0, 2, 4, 32]
[32, 24, 19, 55]
[0, 2, 8, 28]
[32, 24, 19, 55]
[0, 2, 8, 31]
[32, 24, 19, 55]
[0, 5, 3, 1]
[32, 24, 19, 55]
[0, 5, 3, 26]
[32, 24, 19, 55]
[1, 3, 5, 33]
[32, 24, 19, 55]
[1, 3, 26, 7]
[32, 24, 19, 55]
[1, 3, 26, 25]
[32, 24, 19, 55]
[1, 4, 2, 8]
[32, 24, 19, 55]
[1, 4, 32, 24]
[32, 24, 19, 55]
[2, 0, 5, 3]
[32, 24, 19, 55]
[2, 0, 5, 33]
[32, 24, 19, 55]
[2, 4, 1, 3]
[32, 24, 19, 55]
[2, 4, 1, 35]
[32, 24, 19, 55]
[2, 4, 32, 24]
[32, 24, 19, 55]
[2, 8, 28, 22]
[32, 24, 19, 55]
[2, 8, 28, 69]
[32, 24, 19, 55]
[3, 1, 4, 32]
[32, 24, 19, 55]
[3, 5, 0, 34]
[32, 24, 19, 55]
[3, 26, 7, 27]
[32, 24, 19, 55]
[3, 26, 7, 30]
[32, 24, 19, 55]
[3, 26, 25, 6]
[32, 24, 19, 55]
[4, 1, 3, 5]
[32, 24, 19, 55]
[4, 1, 3, 26]
[32, 24, 19, 55]
[4, 2, 0, 5]
[32, 24, 19, 55]
[4, 2, 0, 34]
[32, 24, 19, 55]
[4, 2, 8, 28]
[32, 24, 19, 55]
[4, 2, 8, 31]
[32, 24, 19, 55]
[4, 32, 24, 18]
[32, 24, 19, 55]
[4, 32, 24, 19]
[32, 24, 19, 55]
[4, 32, 24, 68]
[32, 24, 19, 55]
[

[0, 2, 5, 3]
[4, 18, 21, 24]
[0, 2, 5, 20]
[4, 18, 21, 24]
[0, 4, 1, 3]
[4, 18, 21, 24]
[0, 4, 1, 31]
[4, 18, 21, 24]
[0, 4, 18, 12]
[4, 18, 21, 24]
[0, 4, 18, 13]
[4, 18, 21, 24]
[0, 4, 18, 21]
[4, 18, 21, 24]
[1, 3, 5, 2]
[4, 18, 21, 24]
[1, 3, 5, 20]
[4, 18, 21, 24]
[1, 4, 0, 2]
[4, 18, 21, 24]
[1, 4, 0, 30]
[4, 18, 21, 24]
[1, 4, 18, 12]
[4, 18, 21, 24]
[1, 4, 18, 13]
[4, 18, 21, 24]
[1, 4, 18, 21]
[4, 18, 21, 24]
[2, 0, 4, 18]
[4, 18, 21, 24]
[2, 5, 3, 33]
[4, 18, 21, 24]
[2, 5, 20, 51]
[4, 18, 21, 24]
[2, 5, 20, 52]
[4, 18, 21, 24]
[2, 5, 20, 53]
[4, 18, 21, 24]
[3, 1, 4, 18]
[4, 18, 21, 24]
[3, 5, 2, 32]
[4, 18, 21, 24]
[3, 5, 20, 51]
[4, 18, 21, 24]
[3, 5, 20, 52]
[4, 18, 21, 24]
[3, 5, 20, 53]
[4, 18, 21, 24]
[4, 0, 2, 5]
[4, 18, 21, 24]
[4, 0, 2, 32]
[4, 18, 21, 24]
[4, 1, 3, 5]
[4, 18, 21, 24]
[4, 1, 3, 33]
[4, 18, 21, 24]
[4, 18, 12, 13]
[4, 18, 21, 24]
[4, 18, 12, 40]
[4, 18, 21, 24]
[4, 18, 12, 41]
[4, 18, 21, 24]
[4, 18, 13, 12]
[4, 18, 21, 24]
[4, 18, 13, 42]
[4, 18, 21

[0, 2, 4, 1]
[2, 4, 11, 8]
[0, 2, 4, 11]
[2, 4, 11, 8]
[0, 2, 6, 5]
[2, 4, 11, 8]
[0, 2, 6, 13]
[2, 4, 11, 8]
[0, 3, 7, 5]
[2, 4, 11, 8]
[0, 3, 7, 14]
[2, 4, 11, 8]
[0, 3, 9, 18]
[2, 4, 11, 8]
[0, 3, 9, 19]
[2, 4, 11, 8]
[0, 3, 9, 20]
[2, 4, 11, 8]
[1, 4, 2, 6]
[2, 4, 11, 8]
[1, 4, 11, 8]
[2, 4, 11, 8]
[1, 4, 11, 24]
[2, 4, 11, 8]
[1, 4, 11, 25]
[2, 4, 11, 8]
[1, 13, 6, 2]
[2, 4, 11, 8]
[1, 13, 6, 5]
[2, 4, 11, 8]
[2, 0, 3, 7]
[2, 4, 11, 8]
[2, 0, 3, 9]
[2, 4, 11, 8]
[2, 4, 1, 13]
[2, 4, 11, 8]
[2, 4, 1, 17]
[2, 4, 11, 8]
[2, 4, 11, 8]
[2, 4, 11, 8]
Match!
[2, 4, 11, 24]
[2, 4, 11, 8]
[2, 4, 11, 25]
[2, 4, 11, 8]
[2, 6, 5, 7]
[2, 4, 11, 8]
[2, 6, 5, 10]
[2, 4, 11, 8]
[3, 0, 2, 4]
[2, 4, 11, 8]
[3, 0, 2, 6]
[2, 4, 11, 8]
[3, 7, 5, 6]
[2, 4, 11, 8]
[3, 7, 5, 10]
[2, 4, 11, 8]
[3, 7, 14, 26]
[2, 4, 11, 8]
[4, 1, 13, 6]
[2, 4, 11, 8]
[4, 2, 0, 16]
[2, 4, 11, 8]
[4, 2, 6, 5]
[2, 4, 11, 8]
[4, 2, 6, 13]
[2, 4, 11, 8]
[4, 11, 8, 12]
[2, 4, 11, 8]
[4, 11, 8, 15]
[2, 4, 11, 8]
[5, 7, 3, 9]
[2, 

...

[0, 1, 7, 5]
[13, 29, 32, 15]
[0, 1, 7, 9]
[13, 29, 32, 15]
[0, 2, 8, 5]
[13, 29, 32, 15]
[0, 2, 8, 17]
[13, 29, 32, 15]
[1, 0, 2, 8]
[13, 29, 32, 15]
[1, 0, 2, 37]
[13, 29, 32, 15]
[1, 7, 5, 8]
[13, 29, 32, 15]
[1, 7, 5, 40]
[13, 29, 32, 15]
[1, 7, 9, 13]
[13, 29, 32, 15]
[1, 7, 9, 15]
[13, 29, 32, 15]
[2, 0, 1, 7]
[13, 29, 32, 15]
[2, 0, 1, 36]
[13, 29, 32, 15]
[2, 8, 5, 7]
[13, 29, 32, 15]
[2, 8, 5, 40]
[13, 29, 32, 15]
[2, 8, 17, 28]
[13, 29, 32, 15]
[2, 8, 17, 31]
[13, 29, 32, 15]
[3, 4, 12, 6]
[13, 29, 32, 15]
[3, 4, 12, 34]
[13, 29, 32, 15]
[3, 10, 11, 6]
[13, 29, 32, 15]
[3, 10, 11, 19]
[13, 29, 32, 15]
[3, 10, 21, 20]
[13, 29, 32, 15]
[3, 10, 21, 26]
[13, 29, 32, 15]
[3, 10, 21, 46]
[13, 29, 32, 15]
[4, 3, 10, 11]
[13, 29, 32, 15]
[4, 3, 10, 21]
[13, 29, 32, 15]
[4, 12, 6, 11]
[13, 29, 32, 15]
[4, 12, 6, 41]
[13, 29, 32, 15]
[4, 12, 34, 27]
[13, 29, 32, 15]
[5, 7, 1, 36]
[13, 29, 32, 15]
[5, 7, 9, 13]
[13, 29, 32, 15]
[5, 7, 9, 15]
[13, 29, 32, 15]
[5, 8, 2, 37]
[13, 29, 32, 1

[0, 1, 3, 5]
[6, 14, 15, 7]
[0, 1, 3, 10]
[6, 14, 15, 7]
[0, 2, 4, 5]
[6, 14, 15, 7]
[0, 2, 4, 11]
[6, 14, 15, 7]
[1, 0, 2, 4]
[6, 14, 15, 7]
[1, 0, 2, 22]
[6, 14, 15, 7]
[1, 3, 5, 4]
[6, 14, 15, 7]
[1, 3, 5, 15]
[6, 14, 15, 7]
[1, 3, 10, 27]
[6, 14, 15, 7]
[1, 3, 10, 28]
[6, 14, 15, 7]
[1, 3, 10, 29]
[6, 14, 15, 7]
[2, 0, 1, 3]
[6, 14, 15, 7]
[2, 0, 1, 21]
[6, 14, 15, 7]
[2, 4, 5, 3]
[6, 14, 15, 7]
[2, 4, 5, 15]
[6, 14, 15, 7]
[2, 4, 11, 30]
[6, 14, 15, 7]
[2, 4, 11, 31]
[6, 14, 15, 7]
[2, 4, 11, 32]
[6, 14, 15, 7]
[3, 1, 0, 20]
[6, 14, 15, 7]
[3, 5, 4, 11]
[6, 14, 15, 7]
[3, 5, 15, 7]
[6, 14, 15, 7]
[3, 5, 15, 14]
[6, 14, 15, 7]
[4, 2, 0, 20]
[6, 14, 15, 7]
[4, 5, 3, 10]
[6, 14, 15, 7]
[4, 5, 15, 7]
[6, 14, 15, 7]
[4, 5, 15, 14]
[6, 14, 15, 7]
[5, 3, 1, 21]
[6, 14, 15, 7]
[5, 3, 10, 27]
[6, 14, 15, 7]
[5, 3, 10, 28]
[6, 14, 15, 7]
[5, 3, 10, 29]
[6, 14, 15, 7]
[5, 4, 2, 22]
[6, 14, 15, 7]
[5, 4, 11, 30]
[6, 14, 15, 7]
[5, 4, 11, 31]
[6, 14, 15, 7]
[5, 4, 11, 32]
[6, 14, 15, 7]
[5, 15

[0, 2, 6, 3]
[20, 24, 19, 23]
[0, 2, 6, 24]
[20, 24, 19, 23]
[0, 5, 1, 3]
[20, 24, 19, 23]
[0, 5, 1, 27]
[20, 24, 19, 23]
[0, 5, 17, 9]
[20, 24, 19, 23]
[0, 5, 17, 41]
[20, 24, 19, 23]
[1, 3, 6, 2]
[20, 24, 19, 23]
[1, 3, 6, 24]
[20, 24, 19, 23]
[1, 5, 0, 2]
[20, 24, 19, 23]
[1, 5, 0, 26]
[20, 24, 19, 23]
[1, 5, 17, 9]
[20, 24, 19, 23]
[1, 5, 17, 41]
[20, 24, 19, 23]
[2, 0, 5, 17]
[20, 24, 19, 23]
[2, 6, 3, 29]
[20, 24, 19, 23]
[2, 6, 24, 19]
[20, 24, 19, 23]
[2, 6, 24, 20]
[20, 24, 19, 23]
[2, 6, 24, 21]
[20, 24, 19, 23]
[3, 1, 5, 17]
[20, 24, 19, 23]
[3, 6, 2, 28]
[20, 24, 19, 23]
[3, 6, 24, 19]
[20, 24, 19, 23]
[3, 6, 24, 20]
[20, 24, 19, 23]
[3, 6, 24, 21]
[20, 24, 19, 23]
[4, 7, 8, 16]
[20, 24, 19, 23]
[4, 7, 8, 18]
[20, 24, 19, 23]
[4, 15, 9, 16]
[20, 24, 19, 23]
[4, 15, 9, 17]
[20, 24, 19, 23]
[5, 0, 2, 6]
[20, 24, 19, 23]
[5, 0, 2, 28]
[20, 24, 19, 23]
[5, 1, 3, 6]
[20, 24, 19, 23]
[5, 1, 3, 29]
[20, 24, 19, 23]
[5, 17, 9, 15]
[20, 24, 19, 23]
[5, 17, 9, 16]
[20, 24, 19, 23]
[6

[0, 2, 9, 3]
[18, 25, 31, 21]
[0, 2, 9, 25]
[18, 25, 31, 21]
[0, 6, 1, 3]
[18, 25, 31, 21]
[0, 6, 1, 33]
[18, 25, 31, 21]
[0, 6, 13, 8]
[18, 25, 31, 21]
[0, 6, 13, 22]
[18, 25, 31, 21]
[1, 3, 9, 2]
[18, 25, 31, 21]
[1, 3, 9, 25]
[18, 25, 31, 21]
[1, 6, 0, 2]
[18, 25, 31, 21]
[1, 6, 0, 32]
[18, 25, 31, 21]
[1, 6, 13, 8]
[18, 25, 31, 21]
[1, 6, 13, 22]
[18, 25, 31, 21]
[2, 0, 6, 13]
[18, 25, 31, 21]
[2, 9, 3, 35]
[18, 25, 31, 21]
[2, 9, 25, 18]
[18, 25, 31, 21]
[2, 9, 25, 31]
[18, 25, 31, 21]
[3, 1, 6, 13]
[18, 25, 31, 21]
[3, 9, 2, 34]
[18, 25, 31, 21]
[3, 9, 25, 18]
[18, 25, 31, 21]
[3, 9, 25, 31]
[18, 25, 31, 21]
[4, 7, 8, 5]
[18, 25, 31, 21]
[4, 7, 8, 13]
[18, 25, 31, 21]
[4, 7, 12, 14]
[18, 25, 31, 21]
[4, 7, 12, 38]
[18, 25, 31, 21]
[4, 10, 11, 5]
[18, 25, 31, 21]
[4, 10, 11, 30]
[18, 25, 31, 21]
[4, 10, 29, 19]
[18, 25, 31, 21]
[5, 8, 7, 12]
[18, 25, 31, 21]
[5, 8, 13, 6]
[18, 25, 31, 21]
[5, 8, 13, 22]
[18, 25, 31, 21]
[5, 11, 10, 29]
[18, 25, 31, 21]
[5, 11, 30, 20]
[18, 25, 31,

[0, 6, 16, 9]
[5, 17, 25, 22]
[0, 6, 17, 5]
[5, 17, 25, 22]
[0, 6, 17, 25]
[5, 17, 25, 22]
[1, 2, 4, 8]
[5, 17, 25, 22]
[1, 2, 4, 29]
[5, 17, 25, 22]
[1, 3, 7, 8]
[5, 17, 25, 22]
[1, 3, 7, 10]
[5, 17, 25, 22]
[2, 1, 3, 7]
[5, 17, 25, 22]
[2, 1, 3, 28]
[5, 17, 25, 22]
[2, 4, 8, 7]
[5, 17, 25, 22]
[2, 4, 8, 18]
[5, 17, 25, 22]
[3, 1, 2, 4]
[5, 17, 25, 22]
[3, 1, 2, 27]
[5, 17, 25, 22]
[3, 7, 8, 4]
[5, 17, 25, 22]
[3, 7, 8, 18]
[5, 17, 25, 22]
[3, 7, 10, 11]
[5, 17, 25, 22]
[3, 7, 10, 31]
[5, 17, 25, 22]
[3, 7, 10, 32]
[5, 17, 25, 22]
[4, 2, 1, 26]
[5, 17, 25, 22]
[4, 8, 7, 10]
[5, 17, 25, 22]
[4, 8, 18, 11]
[5, 17, 25, 22]
[4, 8, 18, 24]
[5, 17, 25, 22]
[5, 9, 16, 6]
[5, 17, 25, 22]
[5, 9, 24, 18]
[5, 17, 25, 22]
[5, 9, 24, 20]
[5, 17, 25, 22]
[5, 9, 24, 21]
[5, 17, 25, 22]
[5, 17, 6, 16]
[5, 17, 25, 22]
[5, 17, 25, 19]
[5, 17, 25, 22]
[5, 17, 25, 22]
[5, 17, 25, 22]
Match!
[5, 17, 25, 23]
[5, 17, 25, 22]
[6, 16, 9, 24]
[5, 17, 25, 22]
[6, 17, 5, 9]
[5, 17, 25, 22]
[6, 17, 5, 30]
[5, 17,

In [ ]:
import qcportal as ptl
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField


client = ptl.FractalClient()
ds = client.get_collection("TorsionDriveDataset", "OpenFF Fragmenter Phenyl Benchmark")
ds.status("B3LYP-D3", status="COMPLETE")
# output = torsion_barrier_for_molecule(ds.df.iloc[1, 0], ds.df.index[1], True)
# print("Torsionderive record Id, Dihedral_indices, Torsion barrier, oemol")
# print(output)
dihedral_indices = list(ds.df.iloc[1, 0].dict()['keywords']['dihedrals'][0])
print("Dihedral indices for which torsion is driven: ", dihedral_indices)
molecule = Molecule.from_smiles(ds.df.index[1])
display(molecule.visualize)
topology = Topology.from_molecules([molecule])
# Let's label using the Parsley force field
forcefield = ForceField('openff-1.2.1.offxml')
# Run the molecule labeling
molecule_force_list = forcefield.label_molecules(topology)
# Print out a formatted description of the torsion parameters applied to this molecule
for mol_idx, mol_forces in enumerate(molecule_force_list):
    print(f'Forces for molecule {mol_idx}')
    for force_tag, force_dict in mol_forces.items():
        if force_tag == 'ProperTorsions':
            print(f"\n{force_tag}:")
            for (atom_indices, parameter) in force_dict.items():
                atomstr=''
                for idx in atom_indices:
                    atomstr += '%3s' % idx
                print("atoms: %s  parameter_id: %s  smirks %s" % (atomstr, parameter.id, parameter.smirks) )